In [1]:
import os.path
import sys
sys.path.append("../../")

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

import ThinLens.Elements as Elements
import ThinLens.Maps as Maps
from ThinLens.Models import F0D0Model, SIS18_Cell_minimal, SIS18_Cell, \
    SIS18_Lattice_minimal, SIS18_Lattice


general settings

In [2]:
dim = 6
slices = 10
quadSliceMultiplicity = 1
dtype = torch.double
device = torch.device("cpu")
outputPerElement = False  # exceeds outputAtBPM
outputAtBPM = True

# prepare models
Lattice = SIS18_Lattice

startK1f = 3.18468e-01
startK1d = -4.80320e-01
model = Lattice(k1f=startK1f, k1d=startK1d, dim=dim, slices=slices, quadSliceMultiplicity=quadSliceMultiplicity,
                dtype=dtype, cellsIdentical=False).to(device)

# workingpoint: q1=4.2, q2=3.3
perturbedModel = Lattice(dim=dim, slices=slices, quadSliceMultiplicity=quadSliceMultiplicity,
                         dtype=dtype).to(device)

model.requires_grad_(False)
perturbedModel.requires_grad_(False)

# dump off
fileName = "trainDump/{}_q1={},q2={}.json".format(type(model).__name__, startK1f, startK1d)

if not os.path.isfile(fileName):
    fPath = os.path.abspath(fileName)
    print(fPath)

    with open(fileName, "w") as file:
        model.dumpJSON(file)
else:
    raise IOError()

/home/dylan/ThesisWorkspace/Tracking/Playground/GroundThinLens/AverageTrainResults/trainDump/SIS18_Lattice_q1=0.318468,q2=-0.48032.json


create training data

In [3]:
from GroundThinLens.SampleBeam import Beam

# train set
if dim == 6:
    beam = Beam(mass=18.798, energy=19.0, exn=1.258e-6, eyn=2.005e-6, sigt=0.00, sige=0.000, particles=500)
    bunch = beam.bunch[:].to(device)
    label = perturbedModel(bunch, outputPerElement=outputPerElement, outputAtBPM=outputAtBPM).to(device)

trainSet = torch.utils.data.TensorDataset(bunch, label)
trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=25,
                                          shuffle=True, num_workers=2)

prepare training

In [4]:
# activate gradients on kick maps
for m in model.modules():
    if type(m) is Elements.Quadrupole:
        for mod in m.modules():
            if type(mod) is Maps.QuadKick:
                mod.requires_grad_(True)

# set up optimizer
# optimizer = optim.Adam(model.parameters(), lr=1e-4)
optimizer = optim.Adam(model.parameters(), lr=5e-5)
# optimizer = optim.Adam(model.parameters(), lr=5e-6)

# loss function
criterion = nn.MSELoss()

train model

In [5]:
import time

def trainLoop(model, criterion, optimizer, epochs: int):
    # train loop

    for epoch in range(epochs):
        for i, data in enumerate(trainLoader, 0):
            bunch, label = data[0], data[1]

            optimizer.zero_grad()

            out = model(bunch, outputPerElement=outputPerElement, outputAtBPM=outputAtBPM)

            # loss = criterion(out, label)  # full phase-space
            loss = criterion(out[:, [0, 2], :], label[:, [0, 2], :])  # only x-, y-plane

            loss.backward()
            optimizer.step()

        print(loss.item())

    return

# train
t0 = time.time()

trainLoop(model, criterion, optimizer, int(1e0))

print("training completed within {:.2f}s".format(time.time() - t0))

9.57142092762451e-12
training completed within 23.40s


save results

In [6]:
# dump off
with open(fileName, "w") as file:
    model.dumpJSON(file)

compare tunes

In [7]:
import tools.madX

print("perturbed model tunes: {}".format(perturbedModel.getTunes()))
print("trained model tunes: {}".format(model.getTunes()))
print("perturbed model tunes from Mad-X: {}".format(tools.madX.tune(perturbedModel.madX())))
print("trained model tunes from Mad-X: {}".format(tools.madX.tune(model.madX())))

perturbed model tunes: [0.22577502984981296, 0.299027622832203]
trained model tunes: [0.28926923412141536, 0.3216291744221588]
perturbed model tunes from Mad-X: [4.225775029849816, 3.299027622832212]
trained model tunes from Mad-X: [4.289269234121411, 3.321629174422164]


estimate magnitude of beta-beating

In [20]:
import numpy as np

twiss, idealTwiss = tools.madX.twissTable(model.madX()), tools.madX.twissTable(perturbedModel.madX())

beta, idealBeta = twiss["betx"], idealTwiss["betx"]

relDiff = (beta - idealBeta) / idealBeta
maxBetaBeat = np.max(np.abs(relDiff)) * 100  # convert to percent
print("{:.2f}%".format(maxBetaBeat))

31.02%


identify largest deviation from ideal multipole settings

In [23]:
weights, idealWeights = list(), list()
for e in range(len(model.elements)):
    for m in range(len(model.elements[e].maps)):
        if model.elements[e].maps[m].weight.requires_grad:
            weights.append(model.elements[e].maps[m].weight.item())
            idealWeights.append(perturbedModel.elements[e].maps[m].weight.item())

weights, idealWeights = np.array(weights), np.array(idealWeights)
relWeightDiff = (weights - idealWeights) / idealWeights
maxWeightDeviation = np.max(np.abs(relWeightDiff)) * 100  # convert to percent
print("{:.2f}%".format(maxWeightDeviation))

1.93%
